In [1]:
from datetime import datetime
import requests
import json
import time

# library for importing BLS statistics through the BLS.gov API: https://github.com/OliverSherouse/bls
import bls
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account

# import ETL functions
import etl_functions as etl

In [2]:
start = time.time()

In [3]:
# Connect to Google BigQuery
key_path = ''

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"])

client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [4]:
###########
# Extract #
###########

In [5]:
# Extract Chicago crime data
chicago_crime_data = etl.extract_chicago_crime_data()

100 crime records extracted from Google BigQuery public datasets: Chicago Crime Data


In [6]:
# Extract graduation data
cps_graduation_data = etl.extract_cps_graduation_data()

Chicago Public Schools graduation data successfully extracted.


In [7]:
# Extract unemployment data
chicago_unemployment_rate_data = etl.extract_chicago_unemployment_rate_data()

Chicago Unemployment rate data successfully extracted.


In [8]:
# Extract weather data
chicago_weather_data = etl.extract_chicago_weather_data()

Extracting temperature data for 2011.
Extracting temperature data for 2012.
Extracting temperature data for 2013.
Extracting temperature data for 2014.
Extracting temperature data for 2015.
Extracting temperature data for 2016.
Extracting temperature data for 2017.
Extracting temperature data for 2018.
Extracting temperature data for 2019.
Extracting precipitation data for 2011.
Extracting precipitation data for 2012.
Extracting precipitation data for 2013.
Extracting precipitation data for 2014.
Extracting precipitation data for 2015.
Extracting precipitation data for 2016.
Extracting precipitation data for 2017.
Extracting precipitation data for 2018.
Extracting precipitation data for 2019.


In [9]:
##################
# DATA PROFILING #
##################

In [10]:
# Profile Chicago crime data
chicago_crime_data_profile = etl.data_profiling_dataframe(chicago_crime_data)
print(chicago_crime_data_profile)

             column_name        col_data_type col_memory %_of_total_memory  \
0                 arrest                 bool   0.000095          0.232964   
1               domestic                 bool   0.000095          0.232964   
2            description               object   0.007874         19.233547   
3           primary_type               object   0.006778         16.556785   
4               fbi_code               object   0.005647         13.793826   
5   location_description               object    0.00643         15.706465   
6         community_area                int64   0.000763          1.863716   
7               district                int64   0.000763          1.863716   
8                   ward                int64   0.000763          1.863716   
9                   beat                int64   0.000763          1.863716   
10                 block               object   0.007152         17.470006   
11              latitude              float64   0.000763        

In [11]:
# Profile graduation data
cps_graduation_data_profile = etl.data_profiling_dataframe(cps_graduation_data)
print(cps_graduation_data_profile)

  column_name col_data_type col_memory %_of_total_memory non_null_values  \
0   grad_rate       float64   0.000069             100.0               9   

  %_of_non_nulls null_values %_of_nulls unique_values_count count   mean  \
0          100.0           0        0.0                   9     9  67.37   

    std   min   25%   50%   75%   max  
0  8.75  54.2  59.1  68.2  74.7  78.0  


In [12]:
# Profile unemployment data
chicago_unemployment_rate_data_profile = etl.data_profiling_dataframe(chicago_unemployment_rate_data)
print(chicago_unemployment_rate_data_profile)

         column_name col_data_type col_memory %_of_total_memory  \
0  unemployment_rate       float64   0.000824             100.0   

  non_null_values %_of_non_nulls null_values %_of_nulls unique_values_count  \
0             108          100.0           0        0.0                  58   

  count  mean   std  min   25%   50%  75%   max  
0   108  7.35  2.48  3.3  5.18  6.85  9.8  11.7  


In [13]:
# Profile weather data
chicago_weather_data_profile = etl.data_profiling_dataframe(chicago_weather_data)
print(chicago_weather_data_profile)

  column_name col_data_type col_memory %_of_total_memory non_null_values  \
0        temp       float64   0.024162              50.0            3167   
1        prcp       float64   0.024162              50.0            3167   

  %_of_non_nulls null_values %_of_nulls unique_values_count count   mean  \
0          100.0           0        0.0                  99  3167  44.83   
1          100.0           0        0.0                 160  3167   1.19   

     std    min    25%    50%    75%    max  
0  19.13 -23.08  30.92  44.96  60.98  84.92  
1   3.20   0.00   0.00   0.00   0.59  55.20  


In [14]:
#############
# TRANSFORM #
#############

In [15]:
# Clean Chicago crime data
chicago_crime_data = etl.clean_chicago_crime_data(chicago_crime_data)

Cleaning Chicago Crime data for the CRIME_INCIDENT fact.
8 null values dropped.
Chicago crime data has no duplicate rows.
Chicago crime data cleaning successful.


In [16]:
# Clean graduation data
cps_graduation_data = etl.clean_cps_graduation_data(cps_graduation_data)

Cleaning Chicago Public Schools graduation data for the GRADUATION_RATE fact.
Chicago Public Schools graduation data has no null values.
Chicago Public Schools graduation data has no duplicate rows.
Chicago Public Schools graduation data cleaning successful.


In [17]:
# Clean unemployment data
chicago_unemployment_rate_data = etl.clean_chicago_unemployment_rate_data(chicago_unemployment_rate_data)

Cleaning unemployment data for the CHICAGO_UNEMPLOYMENT fact.
Chicago unemployment data has no null values.
Chicago unemployement data has no duplicate rows.
Chicago unemployement data cleaning successful.


In [18]:
# Clean weather data
chicago_weather_data = etl.clean_chicago_weather_data(chicago_weather_data)

Cleaning weather data for the WEATHER fact.
Chicago weather data has no null values.
Chicago weather data has no duplicate rows.
Chicago weather data cleaning successful.


In [19]:
# Create date dimension
date_dim = etl.create_date_dimension()
print(date_dim.head())

Date dimension created.
    date_id   full_date  year  year_week  year_day  fiscal_year fiscal_qtr  \
0  20050101  2005-01-01  2005          0         1         2005          1   
1  20050102  2005-01-02  2005          1         2         2005          1   
2  20050103  2005-01-03  2005          1         3         2005          1   
3  20050104  2005-01-04  2005          1         4         2005          1   
4  20050105  2005-01-05  2005          1         5         2005          1   

   month month_name week_day   day_name  day_is_weekday  
0      1    January        6   Saturday               0  
1      1    January        0     Sunday               0  
2      1    January        1     Monday               1  
3      1    January        2    Tuesday               1  
4      1    January        3  Wednesday               1  


In [20]:
# Create crime_code dimension
crime_code_dim = etl.create_crime_code_dimension(chicago_crime_data)
print(crime_code_dim.head())

Crime code dimension created.
   code_id fbi_code            primary_type
0     1000       14         CRIMINAL DAMAGE
1     1001       18               NARCOTICS
2     1002      04A                 ASSAULT
3     1003       24  PUBLIC PEACE VIOLATION
4     1004       11      DECEPTIVE PRACTICE


In [21]:
# Create location dimension
location_dim = etl.create_location_dimension(chicago_crime_data)
print(location_dim.head())

Location dimension created.
   location_id location_description  community_area  district  ward  beat  \
0         1000            RESIDENCE              62         8    13   813   
1         1001               STREET              62         8    13   813   
2         1002               STREET              34         9    11   915   
3         1003               STREET              52         4    10   432   
4         1004            RESIDENCE               9        16    41  1611   

                   block   latitude  longitude  x_coordinate  y_coordinate  
0    057XX S KILDARE AVE  41.789047 -87.730391     1148678.0     1866296.0  
1        045XX W 59TH ST  41.785803 -87.735969     1147165.0     1865103.0  
2  031XX S PRINCETON AVE  41.837745 -87.634093     1174795.0     1884240.0  
3       095XX S AVENUE M  41.722525 -87.537711     1201448.0     1842483.0  
4       076XX W IBSEN ST  42.005965 -87.818038     1124301.0     1945185.0  


In [22]:
# Create CRIME_INCIDENT fact
crime_incident_fact = etl.create_crime_incident_fact(chicago_crime_data, location_dim, crime_code_dim)
print(crime_incident_fact.head())

    date_id  location_id  code_id  arrest  domestic  \
0  20150923         1000     1000   False     False   
1  20160728         1018     1000   False     False   
2  20170122         1023     1000   False     False   
3  20110822         1092     1000   False     False   
4  20151027         1001     1001    True     False   

                    description  
0           CRIMINAL DEFACEMENT  
1           CRIMINAL DEFACEMENT  
2           CRIMINAL DEFACEMENT  
3           CRIMINAL DEFACEMENT  
4  POSSESSION OF DRUG EQUIPMENT  


In [23]:
# Create CHICAGO_UNEMPLOYMENT fact
chicago_unemployment_fact = etl.create_chicago_unemployment_fact(chicago_unemployment_rate_data, date_dim)
print(chicago_unemployment_fact.head())

    date_id  unemployment_rate
0  20110101               11.0
1  20110102               11.0
2  20110103               11.0
3  20110104               11.0
4  20110105               11.0


In [24]:
# Create GRADUATION_RATE fact
graduation_rate_fact = etl.create_graduation_rate_fact(cps_graduation_data, date_dim)
print(graduation_rate_fact.head())

    date_id  grad_rate
0  20110101       54.2
1  20110102       54.2
2  20110103       54.2
3  20110104       54.2
4  20110105       54.2


In [25]:
# Create WEATHER fact table
weather_fact = etl.create_weather_fact(chicago_weather_data)
print(weather_fact.head())

    date_id   temp     prcp
0  20110101  26.06  4.80315
1  20110102  12.92  0.00000
2  20110103  21.92  0.00000
3  20110104  23.00  0.00000
4  20110105   8.06  0.00000


In [26]:
########
# LOAD #
########

# Load location dimension
etl.load_df_to_bigquery(df = location_dim, table_name = 'location_dim')
# Load crime_code dimension
etl.load_df_to_bigquery(df = crime_code_dim, table_name = 'crime_code_dim')
# Load date dimension
etl.load_df_to_bigquery(df = date_dim, table_name = 'date_dim')

# Load crime_incident fact
etl.load_df_to_bigquery(df = crime_incident_fact, table_name = 'crime_incident_fact')
# Load chicago_unemployment fact
etl.load_df_to_bigquery(df = chicago_unemployment_fact, table_name = 'chicago_unemployment_fact')
# Load graduation_rate fact
etl.load_df_to_bigquery(df = graduation_rate_fact, table_name = 'graduation_rate_fact')
# Load weather fact
etl.load_df_to_bigquery(df = weather_fact, table_name = 'weather_fact')

Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f83e64f79a0>
Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f83e645d2b0>
Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f83e645d520>
Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f83e67a8760>
Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f83e67a8910>
Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f83e67a8d00>
Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7f83e67a8d60>


In [27]:
end = time.time()
print("Total time for ETL in seconds:", end - start)

Total time for ETL: 41.93107891082764
